# ML Pipeline Dev
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database
- Import Python libraries
- Load dataset from the database
- Define feature and target variables X and Y

In [1]:
import re
import pickle
import numpy as np
import pandas as pd

from sqlalchemy import create_engine

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline

from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import RidgeClassifier

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer, classification_report

from xgboost import XGBClassifier

import warnings
warnings.simplefilter('ignore')

In [2]:
engine = create_engine('sqlite:///Data/Database.db')
connection = engine.connect()

In [3]:
df = pd.read_sql_query('''SELECT * FROM TheDRTable''', connection)
df[:2]

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [4]:
X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    """Normalize, tokenize and lemmatize input text string
    
    Arguments:
        text: string  --> input text message
       
    Returns:
    wordsFiltered: list of strings
    """
    
    wordnet_lemmatizer = WordNetLemmatizer()
    stopWords = set(stopwords.words('english'))
    
    # Clean the text
    text = re.sub(pattern = '[^a-zA-Z0-9]', repl = ' ', string = text.lower())
    
    # Extract the word tokens from the provided text
    words = word_tokenize(text)
    
    #Lemmanitizer to remove inflectional and derivationally related forms of a word
    wordsFiltered = [wordnet_lemmatizer.lemmatize(w) for w in words if w not in stopWords]
    
    return wordsFiltered

In [6]:
tokenize(df['message'][7])

['please', 'need', 'tent', 'water', 'silo', 'thank']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset.

In [7]:
pipeline = Pipeline([
    ('vectorize', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('model', MultiOutputClassifier(RandomForestClassifier()))
])

In [8]:
pipeline

Pipeline(memory=None,
     steps=[('vectorize', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        ..._score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None))])

### 4. Training pipeline
- Split data into train and test sets
- Run the training

In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 42)

In [10]:
X_train = np.array(X_train)
X_test = np.array(X_test)
Y_train = np.array(Y_train)
Y_test = np.array(Y_test)

In [11]:
pipeline.fit(X_train, Y_train);

### 5. Test your model
Report the accuracy, f1 score, precision and recall for each output category of the dataset.

In [12]:
Y_pred_train = pipeline.predict(X_train)
Y_pred_test = pipeline.predict(X_test)

In [13]:
def evaluate_by_classification_report(actual, predicted, categories):
    '''
    Evaluate model performance using classification_report function
    
    Arguments:
    actual: numpy array -- Array containing actual labels.
    predicted: numpy array -- Array containing predicted labels.
    categories: list of strings -- List containing names for each of categories.
    
    Output:
    Print accuracy and classfication report for each category
    '''
    
    # Calculate the accuracy for each of them.
    for i in range(len(categories)):
        print("Category:", categories[i],"\n", classification_report(actual[:, i], predicted[:, i]))
        print('Accuracy of %25s: %.2f' %(categories[i], accuracy_score(actual[:, i], predicted[:,i])))

In [14]:
evaluate_by_classification_report(Y_train, Y_pred_train, Y.columns)

Category: related 
               precision    recall  f1-score   support

         0.0       0.99      0.97      0.98      4569
         1.0       0.99      1.00      0.99     15092

   micro avg       0.99      0.99      0.99     19661
   macro avg       0.99      0.98      0.99     19661
weighted avg       0.99      0.99      0.99     19661

Accuracy of                   related: 0.99
Category: request 
               precision    recall  f1-score   support

         0.0       0.99      1.00      0.99     16280
         1.0       1.00      0.93      0.96      3381

   micro avg       0.99      0.99      0.99     19661
   macro avg       0.99      0.96      0.98     19661
weighted avg       0.99      0.99      0.99     19661

Accuracy of                   request: 0.99
Category: offer 
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     19575
         1.0       1.00      0.67      0.81        86

   micro avg       1.00      1.00   

Category: shops 
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     19570
         1.0       1.00      0.68      0.81        91

   micro avg       1.00      1.00      1.00     19661
   macro avg       1.00      0.84      0.90     19661
weighted avg       1.00      1.00      1.00     19661

Accuracy of                     shops: 1.00
Category: aid_centers 
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     19429
         1.0       1.00      0.73      0.84       232

   micro avg       1.00      1.00      1.00     19661
   macro avg       1.00      0.86      0.92     19661
weighted avg       1.00      1.00      1.00     19661

Accuracy of               aid_centers: 1.00
Category: other_infrastructure 
               precision    recall  f1-score   support

         0.0       0.99      1.00      0.99     18787
         1.0       1.00      0.77      0.87       874

   micro avg       

In [15]:
evaluate_by_classification_report(Y_test, Y_pred_test, Y.columns)

Category: related 
               precision    recall  f1-score   support

         0.0       0.63      0.46      0.53      1553
         1.0       0.85      0.92      0.88      5001

   micro avg       0.81      0.81      0.81      6554
   macro avg       0.74      0.69      0.71      6554
weighted avg       0.80      0.81      0.80      6554

Accuracy of                   related: 0.81
Category: request 
               precision    recall  f1-score   support

         0.0       0.90      0.97      0.93      5461
         1.0       0.77      0.44      0.56      1093

   micro avg       0.88      0.88      0.88      6554
   macro avg       0.83      0.70      0.74      6554
weighted avg       0.88      0.88      0.87      6554

Accuracy of                   request: 0.88
Category: offer 
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      6522
         1.0       0.00      0.00      0.00        32

   micro avg       1.00      1.00   

Accuracy of                earthquake: 0.97
Category: cold 
               precision    recall  f1-score   support

         0.0       0.98      1.00      0.99      6432
         1.0       0.83      0.12      0.21       122

   micro avg       0.98      0.98      0.98      6554
   macro avg       0.91      0.56      0.60      6554
weighted avg       0.98      0.98      0.98      6554

Accuracy of                      cold: 0.98
Category: other_weather 
               precision    recall  f1-score   support

         0.0       0.95      1.00      0.98      6235
         1.0       0.62      0.08      0.14       319

   micro avg       0.95      0.95      0.95      6554
   macro avg       0.79      0.54      0.56      6554
weighted avg       0.94      0.95      0.94      6554

Accuracy of             other_weather: 0.95
Category: direct_report 
               precision    recall  f1-score   support

         0.0       0.85      0.97      0.91      5289
         1.0       0.73      0.31   

In [16]:
def evaluation_metrics(actual, predicted, categories):
    """Calculates Accuracy, Precision, Recall and F1_score
    
    Arguments:
    actual: numpy array -- Array containing actual labels.
    predicted: numpy array -- Array containing predicted labels.
    categories: list of strings -- List containing names for each of categories.
       
    Returns:
    metrics_df: dataframe -- Dataframe containing the accuracy, precision, recall and f1 score
    for a given set of actual and predicted labels.
    """
    metrics = []
    
    # Calculate evaluation metrics for each set of labels
    for i in range(len(categories)):
        accuracy = accuracy_score(actual[:, i], predicted[:, i])
        precision = precision_score(actual[:, i], predicted[:, i])
        recall = recall_score(actual[:, i], predicted[:, i])
        f1 = f1_score(actual[:, i], predicted[:, i])
        
        metrics.append([accuracy, precision, recall, f1])
    
    # Create dataframe containing metrics
    metrics = np.array(metrics)
    metrics_df = pd.DataFrame(data = metrics, index = categories, columns = ['Accuracy', 'Precision', 'Recall', 'F1'])
      
    return metrics_df

In [17]:
evaluation_metrics(Y_train, Y_pred_train, Y.columns)

,Accuracy,Precision,Recall,F1
related,0.990845,0.991691,0.996422,0.994051
request,0.987386,0.996828,0.929607,0.962045
offer,0.998576,1.000000,0.674419,0.805556
aid_related,0.983978,0.993582,0.967647,0.980443
medical_help,0.988709,0.999249,0.857603,0.923024
medical_products,0.991811,0.997534,0.835744,0.909500
search_and_rescue,0.994151,0.995536,0.797853,0.885799
security,0.994456,0.995781,0.686047,0.812392
military,0.995372,0.996528,0.865762,0.926554
child_alone,1.000000,0.000000,0.000000,0.000000


In [18]:
evaluation_metrics(Y_test, Y_pred_test, Y.columns)

,Accuracy,Precision,Recall,F1
related,0.809124,0.845686,0.917217,0.880000
request,0.884193,0.770227,0.435499,0.556400
offer,0.995117,0.000000,0.000000,0.000000
aid_related,0.755569,0.756782,0.599259,0.668871
medical_help,0.920812,0.559633,0.114662,0.190328
medical_products,0.951327,0.782609,0.104348,0.184143
search_and_rescue,0.975130,0.600000,0.036364,0.068571
security,0.980165,0.000000,0.000000,0.000000
military,0.970095,0.511628,0.111675,0.183333
child_alone,1.000000,0.000000,0.000000,0.000000


In [19]:
def single_performance_metric(actual, predicted):
    """Weighted F1_score
    
    Arguments:
    actual: numpy array -- Array containing actual labels.
    predicted: numpy array -- Array containing predicted labels.

       
    Returns:
    f1_score: float -- Weighted F1_score.
    """
       
    return f1_score(actual, predicted, average = 'weighted')

In [20]:
single_performance_metric(Y_train, Y_pred_train)

0.9582978919996292

In [21]:
single_performance_metric(Y_test, Y_pred_test)

0.5454345404393559

### 6. Improve your model
Use grid search to find better parameters. 

In [22]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'vectorize', 'tfidf', 'model', 'vectorize__analyzer', 'vectorize__binary', 'vectorize__decode_error', 'vectorize__dtype', 'vectorize__encoding', 'vectorize__input', 'vectorize__lowercase', 'vectorize__max_df', 'vectorize__max_features', 'vectorize__min_df', 'vectorize__ngram_range', 'vectorize__preprocessor', 'vectorize__stop_words', 'vectorize__strip_accents', 'vectorize__token_pattern', 'vectorize__tokenizer', 'vectorize__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'model__estimator__bootstrap', 'model__estimator__class_weight', 'model__estimator__criterion', 'model__estimator__max_depth', 'model__estimator__max_features', 'model__estimator__max_leaf_nodes', 'model__estimator__min_impurity_decrease', 'model__estimator__min_impurity_split', 'model__estimator__min_samples_leaf', 'model__estimator__min_samples_split', 'model__estimator__min_weight_fraction_leaf', 'model__estimator__n_estimators', 'model__estimat

In [23]:
parameters = {'model__estimator__class_weight' : ['balanced'],
              'model__estimator__n_estimators':[100],
              'model__estimator__min_samples_split':[30,40],
              'vectorize__ngram_range': ((1,1),(1,2)),
              'vectorize__max_features': [1000,3000] 
}

tuner = GridSearchCV(pipeline, param_grid = parameters, scoring = make_scorer(single_performance_metric))


tuned_model = tuner.fit(X_train, Y_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] model__estimator__class_weight=balanced, model__estimator__min_samples_split=30, model__estimator__n_estimators=100, vectorize__max_features=1000, vectorize__ngram_range=(1, 1) 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  model__estimator__class_weight=balanced, model__estimator__min_samples_split=30, model__estimator__n_estimators=100, vectorize__max_features=1000, vectorize__ngram_range=(1, 1), score=0.6663820009653014, total= 1.8min
[CV] model__estimator__class_weight=balanced, model__estimator__min_samples_split=30, model__estimator__n_estimators=100, vectorize__max_features=1000, vectorize__ngram_range=(1, 1) 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.2min remaining:    0.0s


[CV]  model__estimator__class_weight=balanced, model__estimator__min_samples_split=30, model__estimator__n_estimators=100, vectorize__max_features=1000, vectorize__ngram_range=(1, 1), score=0.667154640182738, total= 1.7min
[CV] model__estimator__class_weight=balanced, model__estimator__min_samples_split=30, model__estimator__n_estimators=100, vectorize__max_features=1000, vectorize__ngram_range=(1, 1) 


...

[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed: 16.8min remaining:    0.0s


[CV]  model__estimator__class_weight=balanced, model__estimator__min_samples_split=30, model__estimator__n_estimators=100, vectorize__max_features=3000, vectorize__ngram_range=(1, 1), score=0.667794097878998, total= 1.7min
[CV] model__estimator__class_weight=balanced, model__estimator__min_samples_split=30, model__estimator__n_estimators=100, vectorize__max_features=3000, vectorize__ngram_range=(1, 2) 


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 18.9min remaining:    0.0s


[CV]  model__estimator__class_weight=balanced, model__estimator__min_samples_split=30, model__estimator__n_estimators=100, vectorize__max_features=3000, vectorize__ngram_range=(1, 2), score=0.6686107100165224, total= 1.8min
[CV] model__estimator__class_weight=balanced, model__estimator__min_samples_split=30, model__estimator__n_estimators=100, vectorize__max_features=3000, vectorize__ngram_range=(1, 2) 
[CV]  model__estimator__class_weight=balanced, model__estimator__min_samples_split=30, model__estimator__n_estimators=100, vectorize__max_features=3000, vectorize__ngram_range=(1, 2), score=0.6702911449198886, total= 1.7min
[CV] model__estimator__class_weight=balanced, model__estimator__min_samples_split=30, model__estimator__n_estimators=100, vectorize__max_features=3000, vectorize__ngram_range=(1, 2) 
[CV]  model__estimator__class_weight=balanced, model__estimator__min_samples_split=30, model__estimator__n_estimators=100, vectorize__max_features=3000, vectorize__ngram_range=(1, 2), sc

[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed: 49.8min finished


In [24]:
# Best test score
np.max(tuned_model.cv_results_['mean_test_score'])

0.67309455554558

In [25]:
tuned_model.best_params_

{'model__estimator__class_weight': 'balanced',
 'model__estimator__min_samples_split': 40,
 'model__estimator__n_estimators': 100,
 'vectorize__max_features': 3000,
 'vectorize__ngram_range': (1, 1)}

### 7. Test the tuned model
Report the accuracy, f1 score, precision and recall for each output category of the dataset.


In [26]:
Y_pred_train = tuned_model.predict(X_train)
Y_pred_test = tuned_model.predict(X_test)

In [27]:
evaluate_by_classification_report(Y_train, Y_pred_train, Y.columns)

Category: related 
               precision    recall  f1-score   support

         0.0       0.93      0.96      0.95      4569
         1.0       0.99      0.98      0.98     15092

   micro avg       0.97      0.97      0.97     19661
   macro avg       0.96      0.97      0.96     19661
weighted avg       0.97      0.97      0.97     19661

Accuracy of                   related: 0.97
Category: request 
               precision    recall  f1-score   support

         0.0       0.99      0.98      0.98     16280
         1.0       0.90      0.94      0.92      3381

   micro avg       0.97      0.97      0.97     19661
   macro avg       0.95      0.96      0.95     19661
weighted avg       0.97      0.97      0.97     19661

Accuracy of                   request: 0.97
Category: offer 
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     19575
         1.0       0.92      1.00      0.96        86

   micro avg       1.00      1.00   

Accuracy of                     tools: 1.00
Category: hospitals 
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     19438
         1.0       0.94      1.00      0.97       223

   micro avg       1.00      1.00      1.00     19661
   macro avg       0.97      1.00      0.98     19661
weighted avg       1.00      1.00      1.00     19661

Accuracy of                 hospitals: 1.00
Category: shops 
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     19570
         1.0       0.95      1.00      0.97        91

   micro avg       1.00      1.00      1.00     19661
   macro avg       0.97      1.00      0.99     19661
weighted avg       1.00      1.00      1.00     19661

Accuracy of                     shops: 1.00
Category: aid_centers 
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     19429
         1.0       0.95      1.00      0.

In [28]:
evaluate_by_classification_report(Y_test, Y_pred_test, Y.columns)

Category: related 
               precision    recall  f1-score   support

         0.0       0.62      0.58      0.60      1553
         1.0       0.87      0.89      0.88      5001

   micro avg       0.82      0.82      0.82      6554
   macro avg       0.75      0.73      0.74      6554
weighted avg       0.81      0.82      0.81      6554

Accuracy of                   related: 0.82
Category: request 
               precision    recall  f1-score   support

         0.0       0.94      0.93      0.93      5461
         1.0       0.67      0.69      0.68      1093

   micro avg       0.89      0.89      0.89      6554
   macro avg       0.80      0.81      0.81      6554
weighted avg       0.89      0.89      0.89      6554

Accuracy of                   request: 0.89
Category: offer 
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      6522
         1.0       0.00      0.00      0.00        32

   micro avg       1.00      1.00   

Accuracy of                earthquake: 0.97
Category: cold 
               precision    recall  f1-score   support

         0.0       0.99      0.99      0.99      6432
         1.0       0.45      0.47      0.46       122

   micro avg       0.98      0.98      0.98      6554
   macro avg       0.72      0.73      0.72      6554
weighted avg       0.98      0.98      0.98      6554

Accuracy of                      cold: 0.98
Category: other_weather 
               precision    recall  f1-score   support

         0.0       0.97      0.97      0.97      6235
         1.0       0.39      0.33      0.36       319

   micro avg       0.94      0.94      0.94      6554
   macro avg       0.68      0.65      0.66      6554
weighted avg       0.94      0.94      0.94      6554

Accuracy of             other_weather: 0.94
Category: direct_report 
               precision    recall  f1-score   support

         0.0       0.90      0.90      0.90      5289
         1.0       0.58      0.59   

In [29]:
evaluation_metrics(Y_train, Y_pred_train, Y.columns)

,Accuracy,Precision,Recall,F1
related,0.974365,0.989202,0.977273,0.983201
request,0.972840,0.903144,0.943212,0.922743
offer,0.999644,0.924731,1.000000,0.960894
aid_related,0.965566,0.953570,0.963971,0.958742
medical_help,0.975281,0.781415,0.953608,0.858967
medical_products,0.990743,0.861213,0.967975,0.911479
search_and_rescue,0.996643,0.906096,0.983900,0.943396
security,0.994965,0.784223,0.982558,0.872258
military,0.983470,0.673868,0.987934,0.801223
child_alone,1.000000,0.000000,0.000000,0.000000


In [30]:
evaluation_metrics(Y_test, Y_pred_test, Y.columns)

,Accuracy,Precision,Recall,F1
related,0.816753,0.871239,0.891622,0.881312
request,0.891669,0.669319,0.692589,0.680755
offer,0.995117,0.000000,0.000000,0.000000
aid_related,0.772353,0.706284,0.765926,0.734897
medical_help,0.915624,0.482759,0.552632,0.515337
medical_products,0.950412,0.533557,0.460870,0.494557
search_and_rescue,0.974519,0.484375,0.187879,0.270742
security,0.972231,0.133333,0.078740,0.099010
military,0.963381,0.425606,0.624365,0.506173
child_alone,1.000000,0.000000,0.000000,0.000000


In [31]:
single_performance_metric(Y_train, Y_pred_train)

0.9452965502346238

In [32]:
single_performance_metric(Y_test, Y_pred_test)

0.6741174718233907

### 8. Try improving your model further
* Try other machine learning algorithms

In [33]:
dev_pipeline = Pipeline([
    ('vectorize', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('model', MultiOutputClassifier(RidgeClassifier()))
])

In [34]:
dev_pipeline.fit(X_train, Y_train);

In [35]:
Y_pred_test = dev_pipeline.predict(X_train)
Y_pred_test = dev_pipeline.predict(X_test)

In [36]:
print(classification_report(Y_train, Y_pred_train, target_names=Y.columns))

                        precision    recall  f1-score   support

               related       0.99      0.98      0.98     15092
               request       0.90      0.94      0.92      3381
                 offer       0.92      1.00      0.96        86
           aid_related       0.95      0.96      0.96      8160
          medical_help       0.78      0.95      0.86      1552
      medical_products       0.86      0.97      0.91       968
     search_and_rescue       0.91      0.98      0.94       559
              security       0.78      0.98      0.87       344
              military       0.67      0.99      0.80       663
           child_alone       0.00      0.00      0.00         0
                 water       0.95      0.97      0.96      1264
                  food       0.92      0.98      0.95      2200
               shelter       0.89      0.98      0.93      1724
              clothing       0.77      1.00      0.87       310
                 money       0.81      

In [37]:
print(classification_report(Y_test, Y_pred_test, target_names=Y.columns))

                        precision    recall  f1-score   support

               related       0.85      0.93      0.89      5001
               request       0.81      0.56      0.66      1093
                 offer       0.00      0.00      0.00        32
           aid_related       0.75      0.66      0.70      2700
          medical_help       0.62      0.25      0.36       532
      medical_products       0.70      0.28      0.40       345
     search_and_rescue       0.78      0.13      0.22       165
              security       0.50      0.01      0.02       127
              military       0.55      0.23      0.33       197
           child_alone       0.00      0.00      0.00         0
                 water       0.76      0.54      0.63       408
                  food       0.84      0.65      0.73       723
               shelter       0.79      0.49      0.60       590
              clothing       0.76      0.43      0.55        95
                 money       0.67      

In [38]:
dev_pipeline = Pipeline([
    ('vectorize', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('model', MultiOutputClassifier(MultinomialNB()))
])

In [39]:
dev_pipeline.fit(X_train, Y_train);

In [40]:
Y_pred_test = dev_pipeline.predict(X_train)
Y_pred_test = dev_pipeline.predict(X_test)

In [41]:
print(classification_report(Y_train, Y_pred_train, target_names=Y.columns))

                        precision    recall  f1-score   support

               related       0.99      0.98      0.98     15092
               request       0.90      0.94      0.92      3381
                 offer       0.92      1.00      0.96        86
           aid_related       0.95      0.96      0.96      8160
          medical_help       0.78      0.95      0.86      1552
      medical_products       0.86      0.97      0.91       968
     search_and_rescue       0.91      0.98      0.94       559
              security       0.78      0.98      0.87       344
              military       0.67      0.99      0.80       663
           child_alone       0.00      0.00      0.00         0
                 water       0.95      0.97      0.96      1264
                  food       0.92      0.98      0.95      2200
               shelter       0.89      0.98      0.93      1724
              clothing       0.77      1.00      0.87       310
                 money       0.81      

In [42]:
print(classification_report(Y_test, Y_pred_test, target_names=Y.columns))

                        precision    recall  f1-score   support

               related       0.78      0.99      0.87      5001
               request       0.85      0.23      0.36      1093
                 offer       0.00      0.00      0.00        32
           aid_related       0.74      0.63      0.68      2700
          medical_help       0.33      0.00      0.00       532
      medical_products       0.00      0.00      0.00       345
     search_and_rescue       0.00      0.00      0.00       165
              security       0.00      0.00      0.00       127
              military       0.00      0.00      0.00       197
           child_alone       0.00      0.00      0.00         0
                 water       0.00      0.00      0.00       408
                  food       0.56      0.01      0.03       723
               shelter       0.50      0.00      0.00       590
              clothing       0.00      0.00      0.00        95
                 money       0.00      

In [43]:
dev_pipeline = Pipeline([
    ('vectorize', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('model', MultiOutputClassifier(XGBClassifier()))
])

In [44]:
dev_pipeline.fit(X_train, Y_train);

In [45]:
Y_pred_test = dev_pipeline.predict(X_train)
Y_pred_test = dev_pipeline.predict(X_test)

In [46]:
print(classification_report(Y_train, Y_pred_train, target_names=Y.columns))

                        precision    recall  f1-score   support

               related       0.99      0.98      0.98     15092
               request       0.90      0.94      0.92      3381
                 offer       0.92      1.00      0.96        86
           aid_related       0.95      0.96      0.96      8160
          medical_help       0.78      0.95      0.86      1552
      medical_products       0.86      0.97      0.91       968
     search_and_rescue       0.91      0.98      0.94       559
              security       0.78      0.98      0.87       344
              military       0.67      0.99      0.80       663
           child_alone       0.00      0.00      0.00         0
                 water       0.95      0.97      0.96      1264
                  food       0.92      0.98      0.95      2200
               shelter       0.89      0.98      0.93      1724
              clothing       0.77      1.00      0.87       310
                 money       0.81      

In [47]:
print(classification_report(Y_test, Y_pred_test, target_names=Y.columns))

                        precision    recall  f1-score   support

               related       0.79      0.99      0.88      5001
               request       0.85      0.46      0.60      1093
                 offer       0.00      0.00      0.00        32
           aid_related       0.77      0.54      0.64      2700
          medical_help       0.66      0.18      0.28       532
      medical_products       0.76      0.27      0.40       345
     search_and_rescue       0.66      0.15      0.25       165
              security       0.00      0.00      0.00       127
              military       0.55      0.17      0.26       197
           child_alone       0.00      0.00      0.00         0
                 water       0.81      0.66      0.72       408
                  food       0.82      0.79      0.80       723
               shelter       0.82      0.53      0.64       590
              clothing       0.70      0.40      0.51        95
                 money       0.63      

### 9. Export your model as a pickle file

In [48]:
pickle.dump(tuned_model, open('model/model.pkl', 'wb'))